In [1]:
import pandas_ta as pta, pandas as pd, numpy as np

def add_rsi(entire_df,data_name = 'close',period = 14):
    rsi = pta.rsi(entire_df[data_name], length=period)
    entire_df['rsi'+'_'+str(period)] = rsi
    return entire_df

def add_ma(entire_df, data_name = 'close',period = 20):
    entire_df['mean'+'_'+str(period)] = entire_df[data_name].rolling(window=period).mean()
    return entire_df

def add_ema(df, period: int = 30):
    array = df['close']
    ema = pta.ma("ema", pd.Series(array.astype(float)), length=int(period))
    df['ema'+'_'+str(period)] = ema
    return df

def add_stochastic(df,n = 14,m = 5,t = 5):
    ndays_high = df.high.rolling(window = n,min_periods = 1).max()
    ndays_low = df.low.rolling(window = n,min_periods = 1).min()
    fast_k = ((df.close - ndays_low) / (ndays_high - ndays_low)) * 100
    slow_k = fast_k.ewm(span=t).mean()
    slow_d = slow_k.ewm(span=t).mean()

    df = df.assign(fast_k = fast_k, fast_d = slow_k, slow_k = slow_k, slow_d = slow_d)
    return df

def add_bb(df,length = 20,std = 2):
    currunt_close = df["close"]
    currunt_upper_bollinger_band = pta.bbands(df["close"], length = length, std = std)

    df = df.assign(BBL = currunt_upper_bollinger_band['BBL_'+str(length)+'_2.0'],BBM = currunt_upper_bollinger_band['BBL_'+str(length)+'_2.0'],
    BBU = currunt_upper_bollinger_band['BBL_'+str(length)+'_2.0'],BBP = currunt_upper_bollinger_band['BBL_'+str(length)+'_2.0'])

    return df
def add_kdj(df,high=None, low=None, close=None, length=None, signal=None, offset=None):
    temp = pta.kdj(high=df.high,low=df.low,close=df.close)
    df = pd.concat([df,temp],axis=1)
    return df
def add_macd(
    df: np.ndarray,
    fast_period: int = 12,
    slow_period: int = 26,
):

    temp = pta.macd(
        df.close,
        fast=int(fast_period),
        slow=int(slow_period),
        signal=9,
    )
    df = pd.concat([df,temp],axis=1)
    return df
def add_disparity(df,period = 20):
    # 종가기준으로 이동평균선 값을 구함
    ma = df["close"].rolling(period).mean()

    # 시가가 이평선 기준으로 얼마나 위에 있는지 구함
    df['disparity'] = 100*(df["open"]/ma)
    return df


data와 원하는 파라미터의 이름을 딕셔너리에 넣어서 주면 반복문으로 데이터를 넣어서 준다.
init에서 모든 수행이 일어나고 get_data 함수를 통해 데이터를 반환받을 수 있어야한다.

In [11]:
import pandas_ta as pta, pandas as pd, numpy as np



class DataManage:
    def __init__(self,data,parameter = [
        {"rsi" : {"period" : 14}},
        {"ma" : {"period" : 7}},
        {"ma" : {"period" : 25}},
        {"ema" :{"period" : 7}},
        {"ema" :{"period" : 25}},
        {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 21,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 12, "slow_period" : 26}}
        ],target_data = "close"):

        self.data = data
        self.parameter = parameter
        self.target_data = target_data
        fuction_list = {
            "rsi" : "self.rsi()",
            "ma" : "self.ma()",
            "ema" : "self.add_ema()",
            "stochastic" : "self.stochastic()",
            "bb" : "self.bb()",
            "kdj" : "self.kdj()",
            "macd" : "self.macd()",
            "disparity" : "self.disparity()",
        }
        
        for line in parameter:
            for indicator in line.keys():
                eval("self." + indicator + "("+"line[indicator]"+")")
    def rsi(self,parameter):
        rsi = pta.rsi(self.data[self.target_data], length=parameter["period"])
        self.data.loc[:, 'rsi'+'_'+str(parameter["period"])] = rsi
        
    def ma(self,parameter):
        self.data['mean'+'_'+str(parameter["period"])] = self.data[self.target_data].rolling(window=parameter["period"]).mean()

    def ema(self,parameter):
        array = self.data[self.target_data]
        ema = pta.ma("ema", pd.Series(array.astype(float)), length=int(parameter["period"]))
        self.data['ema'+'_'+str(parameter["period"])] = ema

    def stochastic(self,parameter):
        ndays_high = self.data.high.rolling(window = parameter["n"],min_periods = 1).max()
        ndays_low = self.data.low.rolling(window = parameter["n"],min_periods = 1).min()
        fast_k = ((self.data.close - ndays_low) / (ndays_high - ndays_low)) * 100
        slow_k = fast_k.ewm(span=parameter["t"]).mean()
        slow_d = slow_k.ewm(span=parameter["t"]).mean()

        self.data = self.data.assign(fast_k = fast_k, fast_d = slow_k, slow_k = slow_k, slow_d = slow_d)

    def bb(self,parameter):
        currunt_upper_bollinger_band = pta.bbands(self.data["close"], length = parameter["length"], std = parameter["std"])

        self.data = self.data.assign(BBL = currunt_upper_bollinger_band['BBL_'+str(parameter["length"])+'_2.0'],BBM = currunt_upper_bollinger_band['BBL_'+str(parameter["length"])+'_2.0'],
        BBU = currunt_upper_bollinger_band['BBL_'+str(parameter["length"])+'_2.0'],BBP = currunt_upper_bollinger_band['BBL_'+str(parameter["length"])+'_2.0'])

    def kdj(self,high=None, low=None, close=None, length=None, signal=None, offset=None):
        temp = pta.kdj(high=self.data.high,low=self.data.low,close=self.data.close)
        self.data = pd.concat([self.data,temp],axis=1)
        
    def macd(self,parameter):

        temp = pta.macd(
            self.data.close,
            fast=int(parameter["fast_period"]),
            slow=int(parameter["slow_period"]),
            signal=9,
        )
        self.datga = pd.concat([self.data,temp],axis=1)

    def disparity(self,parameter):
        # 종가기준으로 이동평균선 값을 구함
        ma = self.data["close"].rolling(parameter["period"]).mean()

        # 시가가 이평선 기준으로 얼마나 위에 있는지 구함
        self.data['disparity'] = 100*(self.data["open"]/ma)
        
    def get_data(self):
        return self.data.dropna()

In [12]:
df = pd.read_csv("BTC_USDT_1d.csv",index_col=0)
parameter = [
        {"rsi" : {"period" : 14}},
        {"ma" : {"period" : 7}},
        {"ma" : {"period" : 25}},
        {"ema" :{"period" : 7}},
        {"ema" :{"period" : 25}},
        {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 21,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 12, "slow_period" : 26}}
]
DataManageBot = DataManage(df, parameter = parameter)
data = DataManageBot.get_data()
data.head()

,datetime,open,high,low,close,volume,rsi_14,mean_7,mean_25,ema_7,...,fast_d,slow_k,slow_d,BBL,BBM,BBU,BBP,K_9_3,D_9_3,J_9_3
24,2018-01-25,11349.96,11794.05,10950.21,11175.27,20839.954183,40.074500,11412.772857,13334.7152,11389.911113,...,35.443828,35.443828,33.904945,9045.898460,9045.898460,9045.898460,9045.898460,46.263193,43.042512,52.704555
25,2018-01-26,11184.70,11643.00,10311.15,11089.00,33056.871960,39.612390,11357.632857,13243.0752,11314.683335,...,35.976693,35.976693,34.595546,9067.229340,9067.229340,9067.229340,9067.229340,43.227792,43.104366,43.474643
26,2018-01-27,11089.00,11650.00,10842.69,11491.00,18860.768345,42.915669,11170.641429,13115.7108,11358.762501,...,39.417959,39.417959,36.203045,9278.246568,9278.246568,9278.246568,9278.246568,45.396346,43.869135,48.450767
27,2018-01-28,11499.98,12244.00,11408.00,11879.95,16887.339524,45.993842,11220.634286,12994.1284,11489.059376,...,44.463138,44.463138,38.956442,9484.020059,9484.020059,9484.020059,9484.020059,50.895858,46.212962,60.261649
28,2018-01-29,11879.95,11975.02,11139.55,11251.00,14170.377538,42.045600,11290.770000,12841.7868,11429.544532,...,46.028043,46.028043,41.313661,9510.708773,9510.708773,9510.708773,9510.708773,49.458617,47.295335,53.785180


In [13]:
data.columns

Index(['datetime', 'open', 'high', 'low', 'close', 'volume', 'rsi_14',
       'mean_7', 'mean_25', 'ema_7', 'ema_25', 'fast_k', 'fast_d', 'slow_k',
       'slow_d', 'BBL', 'BBM', 'BBU', 'BBP', 'K_9_3', 'D_9_3', 'J_9_3'],
      dtype='object')

In [4]:
DataManageBot.parameter

{'rsi': {'period': 14},
 'ma': {'period': 25},
 'ema': {'period': 25},
 'stochastic': {'n': 14, 'm': 5, 't': 5},
 'bb': {'length': 21, 'std': 2},
 'kdj': {},
 'macd': {'fast_period': 12, 'slow_period': 26}}

In [9]:
data

,datetime,open,high,low,close,volume,rsi_14,mean_20,ema_30,fast_k,...,slow_k,slow_d,BBL,BBM,BBU,BBP,K_9_3,D_9_3,J_9_3,disparity
29,2018-01-30,11250.11,11308.42,9900.00,10237.51,25554.372946,36.594241,11844.6170,12977.211333,29.538347,...,40.531449,41.052922,9529.024710,9529.024710,9529.024710,9529.024710,37.770479,44.119428,25.072581,94.980783
30,2018-01-31,10230.00,10425.85,9700.00,10285.10,18015.956805,37.007246,11696.9330,12803.526731,17.213886,...,32.758901,38.288239,9379.087889,9379.087889,9379.087889,9379.087889,32.846285,40.360960,17.816935,87.458824
31,2018-02-01,10285.10,10335.00,8750.99,9224.52,33564.764311,32.004081,11471.1585,12572.623071,10.890729,...,25.469494,34.015314,9113.990296,9113.990296,9113.990296,9113.990296,26.415976,35.712011,7.823906,89.660517
32,2018-02-02,9224.52,9250.00,8010.02,8873.03,49971.626975,30.530914,11204.3100,12333.939647,16.958408,...,22.632461,30.221024,8941.300225,8941.300225,8941.300225,8941.300225,24.404887,31.942634,9.329395,82.330103
33,2018-02-03,8873.03,9473.01,8229.00,9199.96,28725.000735,33.592796,10990.5585,12131.747412,24.843312,...,23.369412,27.937151,8820.134422,8820.134422,8820.134422,8820.134422,25.638133,29.841009,17.232381,80.733204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,2023-03-01,23141.57,24000.00,23020.03,23628.97,315287.417370,53.872444,23406.8575,23107.238021,35.876978,...,44.451614,52.605512,21353.970588,21353.970588,21353.970588,21353.970588,31.339940,37.024837,19.970147,98.866625
1886,2023-03-02,23629.76,23796.93,23195.90,23465.32,239315.452190,52.270403,23498.8640,23130.340084,29.403481,...,39.435570,48.215531,21615.701860,21615.701860,21615.701860,21615.701860,34.089643,36.046439,30.176051,100.557031
1887,2023-03-03,23465.32,23476.95,21971.13,22351.32,284500.791880,42.914867,23523.3025,23080.080724,11.595153,...,30.155431,42.195498,21714.496974,21714.496974,21714.496974,21714.496974,27.547883,33.213587,16.216475,99.753510
1888,2023-03-04,22354.34,22410.00,22269.94,22344.16,67939.424020,42.861767,23551.3335,23032.601968,11.376785,...,23.895882,36.095626,21836.295548,21836.295548,21836.295548,21836.295548,24.118641,30.181938,11.992048,94.917513


In [26]:
def a():
    print("a")
dic = {
    "1" : "a()"
}
eval(dic["1"])

a
